In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, cross_val_score, cross_validate, cross_val_predict
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error, median_absolute_error, mean_absolute_percentage_error
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
from scipy.stats import randint
import matplotlib.pyplot as plt
from sklearn.svm import SVR




In [3]:
import numpy as np
import pandas as pd
from scipy import stats

def regression_metrics(y_true, y_pred, n_features=None, verbose=True):
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    n = len(y_true)
    
    # Core errors
    residuals = y_true - y_pred
    mse = np.mean(residuals**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(residuals))
    
    # Variance explained
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - ss_res/ss_tot
    
    # Adjusted R²
    if n_features is not None:
        adj_r2 = 1 - (1-r2) * (n-1)/(n-n_features-1)
    else:
        adj_r2 = None
    
    # Correlations
    pearson_r, pearson_p = stats.pearsonr(y_true, y_pred)
    spearman_r, spearman_p = stats.spearmanr(y_true, y_pred)
    
    # CCC (Lin’s Concordance Correlation Coefficient)
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    cov = np.mean((y_true-mean_true)*(y_pred-mean_pred))
    ccc = (2*cov) / (var_true + var_pred + (mean_true-mean_pred)**2)
    
    # Normalized RMSE
    nrmse = rmse / np.mean(y_true)
    
    # MAPE (safe)
    with np.errstate(divide='ignore', invalid='ignore'):
        mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true==0, np.nan, y_true))) * 100
    
    # Percent within tolerance
    def within_tol(tol):
        return np.mean(np.abs(residuals) <= tol*np.abs(y_true)) * 100
    p10 = within_tol(0.10)
    p15 = within_tol(0.15)
    p20 = within_tol(0.20)
    
    # Bland–Altman
    mean_vals = (y_true + y_pred) / 2
    diff_vals = y_true - y_pred
    bias = np.mean(diff_vals)
    loa_lower = bias - 1.96*np.std(diff_vals)
    loa_upper = bias + 1.96*np.std(diff_vals)
    
    metrics = {
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2,
        "Adjusted_R2": adj_r2,
        "Pearson_r": pearson_r,
        "Pearson_p": pearson_p,
        "Spearman_r": spearman_r,
        "Spearman_p": spearman_p,
        "CCC": ccc,
        "NRMSE": nrmse,
        "MAPE (%)": mape,
        "P10 (%)": p10,
        "P15 (%)": p15,
        "P20 (%)": p20,
        "BlandAltman_bias": bias,
        "BlandAltman_LOA": (loa_lower, loa_upper)
    }
    
    if verbose:
        df = pd.DataFrame(metrics.items(), columns=["Metric", "Value"])
        print(df.to_string(index=False))
    
    return metrics


def print_metrics(metrics):
    print("\n=== Regression Metrics ===")
    for k, v in metrics.items():
        if isinstance(v, tuple):  
            print(f"{k:15s}: ({v[0]:.4f}, {v[1]:.4f})")
        elif isinstance(v, (float, np.float32, np.float64)):
            print(f"{k:15s}: {v:.4f}")
        else:
            print(f"{k:15s}: {v}")

    


In [4]:
df = pd.read_csv("Covid-19vsControls_randomregressor.csv")
df

,Age,Weight,BMI,Days_BTW,Machine,BHT,Altitude,pB,Hb,FEV1_FVC_Ratio,...,KCO_Hb,KCO_Sealevel,KCO_Hb_Sealevel,KNO,FEV1_Zscores,DLCO_Zscores_Aguilaniu,DLCO_Zscores_GLI,DLNO_Zscores_GAMLSS,DLNO_Zscores_Munkholm,DLNODLCOratio
0,-2.485939,-0.725444,-0.987350,-1.158842,0,-0.654247,-0.668745,0.759268,-0.374736,2.437953,...,0.398095,0.474985,0.409281,2.583088,-0.053404,-0.538701,0.670777,1.391156,2.629717,6.07
1,-2.267654,-1.875878,-1.808271,0.196057,0,-0.943663,-0.668745,0.759268,0.716298,2.531730,...,0.267713,0.124107,0.278073,1.436308,-0.130326,-0.944026,-0.127360,0.249336,0.146309,5.51
2,-2.267654,-1.818356,-1.779429,-0.173461,0,-0.999320,-0.668745,0.759268,0.248712,1.852927,...,-0.087875,-0.151583,-0.079766,1.433667,0.391124,-1.131038,-0.374594,0.307515,0.264504,5.81
3,-2.122131,-1.415704,-1.518933,-0.296633,0,-0.019759,-0.668745,0.759268,-0.140943,1.410708,...,0.149184,0.161701,0.158794,0.816012,-1.317352,-1.639099,-1.074642,-0.664322,-1.283252,4.96
4,-2.049370,-1.530747,-1.689478,-0.358220,0,-0.943663,-0.668745,0.759268,-0.374736,2.255752,...,-1.047963,-1.041310,-1.034003,0.697655,-1.944873,-2.086964,-1.825750,-0.446785,-0.896973,6.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,1.370421,0.079860,-0.404561,-0.217451,1,1.583160,0.575129,-0.521569,1.417678,-1.270371,...,-1.154640,-1.291937,-1.165210,-1.521271,0.709337,-0.478504,-0.991335,-1.177306,-0.723973,3.25
837,1.370421,0.770120,-0.109711,0.081683,1,0.158343,0.575129,-0.521569,2.664574,-0.373148,...,-1.142787,-1.417251,-1.141354,-1.309812,-0.509267,-0.566792,-0.967149,-0.901084,-0.604458,3.26
838,1.588705,-0.380313,-0.405592,-0.446200,1,0.047030,0.575129,-0.521569,0.794229,-0.893848,...,0.208448,0.023856,0.194578,-0.039490,-1.085777,0.942945,0.317393,0.670752,0.796710,4.24
839,1.806990,-0.092705,-0.084620,0.240048,1,-0.843481,0.575129,-0.521569,0.560436,-3.896490,...,-1.842110,-2.006225,-1.845104,-1.828852,-2.035560,-0.550740,-1.505958,-1.276969,-0.438721,2.58


In [5]:
selected_features = [col for col in df.columns if col != 'DLNODLCOratio']


In [6]:
y = df['DLNODLCOratio']
X = df[selected_features]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

In [8]:
# kfold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [9]:
#ridge
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

mse = mean_squared_error(y_test, y_pred_ridge)
r2 = r2_score(y_test, y_pred_ridge)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

metrics = regression_metrics(y_test, y_pred_ridge)
print_metrics(metrics)

Mean Squared Error: 0.2700
R² Score: 0.4743
          Metric                                   Value
            RMSE                                0.519569
             MAE                                0.378559
              R2                                0.474329
     Adjusted_R2                                    None
       Pearson_r                                0.693805
       Pearson_p                                     0.0
      Spearman_r                                0.648411
      Spearman_p                                     0.0
             CCC                                0.656192
           NRMSE                                0.117003
        MAPE (%)                                8.700708
         P10 (%)                                67.98419
         P15 (%)                               83.794466
         P20 (%)                               90.513834
BlandAltman_bias                                0.056344
 BlandAltman_LOA (-0.9560060108383553, 1.068

In [10]:
# hyper parameter tuning for ridge 


param_dist = {
    "alpha": np.logspace(-3, 3, 100)  # 100 values between 1e-3 and 1e3
}

random_search = RandomizedSearchCV(
    ridge,
    param_distributions=param_dist,
    n_iter=35,            
    scoring="r2",         
    cv=5,                 
    random_state=42
)

random_search.fit(X_train, y_train)

#print("Best alpha:", random_search.best_params_["alpha"])
#print("Best CV R²:", random_search.best_score_)

y_pred1 = random_search.predict(X_test)


#print("Test R²:", r2_score(y_test, y_pred1))
metrics = regression_metrics(y_test, y_pred1)
print_metrics(metrics)

          Metric                                    Value
            RMSE                                 0.522302
             MAE                                 0.381419
              R2                                 0.468783
     Adjusted_R2                                     None
       Pearson_r                                 0.690168
       Pearson_p                                      0.0
      Spearman_r                                 0.635879
      Spearman_p                                      0.0
             CCC                                 0.653071
           NRMSE                                 0.117619
        MAPE (%)                                 8.777572
         P10 (%)                                 67.98419
         P15 (%)                                83.003953
         P20 (%)                                89.328063
BlandAltman_bias                                 0.057212
 BlandAltman_LOA (-0.960340424389525, 1.0747650018316233)

=== Regressio

#focus on RMSE, MAE, and R².

In [13]:


rf = RandomForestRegressor(
    n_estimators=5000,      # number of trees
    max_depth=30,        
    random_state=42,
    n_jobs=-1,             # use all CPU cores
    min_samples_split=10,
    min_samples_leaf=5
)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
metrics = regression_metrics(y_test, y_pred)
print_metrics(metrics)

          Metric                                     Value
            RMSE                                   0.38754
             MAE                                  0.270689
              R2                                  0.707544
     Adjusted_R2                                      None
       Pearson_r                                  0.850344
       Pearson_p                                       0.0
      Spearman_r                                  0.848868
      Spearman_p                                       0.0
             CCC                                  0.810794
           NRMSE                                  0.087271
        MAPE (%)                                  6.204758
         P10 (%)                                 81.818182
         P15 (%)                                 91.699605
         P20 (%)                                 95.256917
BlandAltman_bias                                  0.037049
 BlandAltman_LOA (-0.7190505474510346, 0.793147880237584

In [12]:
scores = cross_val_score(rf, X, y, cv=kf, scoring='r2')
scores

array([0.72692925, 0.72106154, 0.65838524, 0.79432303, 0.76683094])

In [17]:

scoring = {
    'MAE': 'neg_mean_absolute_error',
    'MSE': 'neg_mean_squared_error',
    'R2': 'r2',
    'MedAE': 'neg_median_absolute_error',
    'MAPE': 'neg_mean_absolute_percentage_error',
    'MaxError': 'max_error',
    'ExplVar': 'explained_variance'
}

# Run CV
cv_results = cross_validate(rf, X, y, cv=kf, scoring=scoring)
y_pred = cross_val_predict(rf, X, y,cv=kf)



metrics = regression_metrics(y, y_pred)
print_metrics(metrics)

          Metric                                    Value
            RMSE                                 0.359151
             MAE                                  0.24474
              R2                                 0.736064
     Adjusted_R2                                     None
       Pearson_r                                 0.864241
       Pearson_p                                      0.0
      Spearman_r                                  0.87173
      Spearman_p                                      0.0
             CCC                                 0.832711
           NRMSE                                 0.081305
        MAPE (%)                                 5.677327
         P10 (%)                                85.612366
         P15 (%)                                 92.74673
         P20 (%)                                96.551724
BlandAltman_bias                                  -0.0004
 BlandAltman_LOA (-0.7043364648299624, 0.703536514603867)

=== Regressio

In [19]:
svr = SVR(kernel='rbf', C=10, gamma='scale', epsilon=0.01)
y_pred_svr = cross_val_predict(svr, X, y, cv=kf)
metrics = regression_metrics(y, y_pred_svr)
print_metrics(metrics)

          Metric                                     Value
            RMSE                                  0.372965
             MAE                                  0.238289
              R2                                   0.71537
     Adjusted_R2                                      None
       Pearson_r                                  0.845834
       Pearson_p                                       0.0
      Spearman_r                                  0.845776
      Spearman_p                                       0.0
             CCC                                  0.832987
           NRMSE                                  0.084433
        MAPE (%)                                   5.64354
         P10 (%)                                 85.255648
         P15 (%)                                  92.74673
         P20 (%)                                 95.719382
BlandAltman_bias                                 -0.003134
 BlandAltman_LOA (-0.7341201757518938, 0.727852210735618

In [20]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(kernel="rbf"))
])
param_grid = {
    "svr__C": [0.1, 1, 10, 100, 1000],
    "svr__gamma": ["scale", 0.001, 0.01, 0.1, 1],
    "svr__epsilon": [0.01, 0.1, 0.2, 0.5, 1]
}
from sklearn.model_selection import GridSearchCV

kf = KFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=kf,
    scoring="r2",   # could also try "neg_mean_squared_error"
    n_jobs=-1,
    verbose=2
)

grid.fit(X, y)

print("Best parameters:", grid.best_params_)
print("Best cross-validated R²:", grid.best_score_)


Fitting 5 folds for each of 125 candidates, totalling 625 fits
Best parameters: {'svr__C': 10, 'svr__epsilon': 0.01, 'svr__gamma': 'scale'}
Best cross-validated R²: 0.7165523434545384


In [23]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(
    loss="squared_error",
    learning_rate=0.1,
    n_estimators=300,
    criterion='friedman_mse',
    min_samples_split=12, 
    min_samples_leaf=3,
    max_depth=7,
    random_state=42,
    max_features='sqrt'
)
#gbr_cv_pred = cross_val_predict(gbr, X, y, cv=kf)
#metrics = regression_metrics(gbr_cv_pred, y)
gbr.fit(X_train, y_train)

y_pred = gbr.predict(X_test)

metrics = regression_metrics(y_test, y_pred)
print_metrics(metrics)



          Metric                                     Value
            RMSE                                  0.364721
             MAE                                  0.257232
              R2                                   0.74097
     Adjusted_R2                                      None
       Pearson_r                                  0.869596
       Pearson_p                                       0.0
      Spearman_r                                  0.880718
      Spearman_p                                       0.0
             CCC                                  0.835641
           NRMSE                                  0.082133
        MAPE (%)                                  5.820701
         P10 (%)                                 84.980237
         P15 (%)                                 92.490119
         P20 (%)                                 96.442688
BlandAltman_bias                                  0.036551
 BlandAltman_LOA (-0.6747039611525066, 0.747805376637235

In [24]:
param_dist = {
    "n_estimators":[100, 200, 300],
    "learning_rate":[0.01, 0.05, 0.1, 0.2],
    "max_depth":[4,5,6,7],
    "min_samples_split":[5, 10, 12, 15],
    "min_samples_leaf":[3,5,7,9],
    "max_features":[None, 'sqrt', 'log2']
}

gbr = GradientBoostingRegressor(random_state=43)

random_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_dist,
    n_iter=75,
    cv=kf,
    scoring='r2',
    random_state=42,
    verbose=1,
    n_jobs=-1
)
random_search.fit(X,y)

print("Best Hyperparameters:")
print(random_search.best_params_)

best_cv_score = random_search.best_score_
print(f"best R2 score {best_cv_score}")


Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best Hyperparameters:
{'n_estimators': 300, 'min_samples_split': 12, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 7, 'learning_rate': 0.1}
best R2 score 0.7845093732654128


In [20]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import numpy as np

# 1️⃣ Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [1, 1.5, 2, 3]
}

# 2️⃣ Initialize the base model
xgbr = XGBRegressor(random_state=42)

# 3️⃣ Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgbr,
    param_distributions=param_grid,
    n_iter=50,           # number of random combinations
    scoring='r2',        # metric to optimize
    cv=5,                # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# 4️⃣ Fit to training data
random_search.fit(X_train, y_train)

# 5️⃣ Best parameters & score
print("Best Parameters:", random_search.best_params_)
print("Best CV R2 Score:", random_search.best_score_)

# 6️⃣ Predict on test set using the best estimator
best_xgbr = random_search.best_estimator_
y_pred_best = best_xgbr.predict(X_test)

# 7️⃣ Evaluate metrics
metrics = regression_metrics(y_pred_best, y_test)
print_metrics(metrics)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'subsample': 0.7, 'reg_lambda': 1.5, 'reg_alpha': 0.01, 'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.6}
Best CV R2 Score: 0.7213854166529368
          Metric                                     Value
            RMSE                                  0.356771
             MAE                                   0.25723
              R2                                  0.548787
     Adjusted_R2                                      None
       Pearson_r                                  0.880623
       Pearson_p                                       0.0
      Spearman_r                                  0.868613
      Spearman_p                                       0.0
             CCC                                  0.840423
           NRMSE                                  0.081161
        MAPE (%)                                  5.892653
         P10 (%)      

In [ ]:

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



from lightgbm import LGBMRegressor
model = LGBMRegressor(
    objective="regression",
    boosting_type="gbdt",
    learning_rate=0.03,
    num_leaves=64,
    max_depth=-1,
    min_data_in_leaf=30,
    feature_fraction=0.85,
    bagging_fraction=0.85,
    bagging_freq=5,
    lambda_l1=0.1,
    lambda_l2=0.2,
    n_estimators=10000,          # large, relies on early stopping
    random_state=42,
    n_jobs=-1
)

model.fit(
    X_train, y_train,
     eval_set=[(X_val, y_val)],
    eval_metric="rmse",
    callbacks = [
        early_stopping(stopping_rounds=150)
        log_evaluation(200)
    ]
)

y_pred = model.predict(X_test)
metrics = regression_metrics(y_pred, y_test)
print_metrics(metrics)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1817485341.py, line 29)

In [26]:

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



from lightgbm import LGBMRegressor, early_stopping, log_evaluation
model = LGBMRegressor(
    objective="regression",
    boosting_type="gbdt",
    learning_rate=0.03,
    num_leaves=64,
    max_depth=-1,
    min_data_in_leaf=30,
    feature_fraction=0.85,
    bagging_fraction=0.85,
    bagging_freq=5,
    lambda_l1=0.1,
    lambda_l2=0.2,
    n_estimators=10000,          # large, relies on early stopping
    random_state=42,
    n_jobs=-1
)

model.fit(
    X_train, y_train,
     eval_set=[(X_val, y_val)],
    eval_metric="rmse",
    callbacks = [
        early_stopping(stopping_rounds=150),
        log_evaluation(200)
    ]
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
metrics = regression_metrics(y_pred, y_test)
print_metrics(metrics)

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.85, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.85
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l2 is set=0.2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2
[LightGBM] [Warning] bagging_fraction is set=0.85, subsample=1.0 will be ignored. Current value: bagging_fraction=0.85
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.85, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.85
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignor

In [28]:
from catboost import CatBoostRegressor

cbr = CatBoostRegressor(
    iterations=10000,          # large number to allow early stopping
    learning_rate=0.05,        # smaller learning rate helps generalization
    depth=7,
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=100, # stop if no improvement for 100 rounds
    l2_leaf_reg=3,             # L2 regularization
    subsample=0.8,             # for regularization
    colsample_bylevel=0.8,
    verbose=100
)
cbr.fit(
    X_train, y_train,
    eval_set = (X_val, y_val),
    use_best_model = True
)

y_pred = cbr.predict(X_test)
metrics = regression_metrics(y_test, y_pred)
print_metrics(metrics)

0:	learn: 0.6978686	test: 0.6423839	best: 0.6423839 (0)	total: 169ms	remaining: 28m 7s
100:	learn: 0.2376107	test: 0.3392953	best: 0.3392953 (100)	total: 611ms	remaining: 59.9s
200:	learn: 0.1448873	test: 0.3074603	best: 0.3074603 (200)	total: 979ms	remaining: 47.7s
300:	learn: 0.0916157	test: 0.2932721	best: 0.2932721 (300)	total: 1.33s	remaining: 43s
400:	learn: 0.0619379	test: 0.2908978	best: 0.2908573 (398)	total: 1.69s	remaining: 40.4s
500:	learn: 0.0428131	test: 0.2883224	best: 0.2883224 (500)	total: 2.04s	remaining: 38.7s
600:	learn: 0.0304152	test: 0.2873885	best: 0.2873470 (586)	total: 2.41s	remaining: 37.7s
700:	learn: 0.0217006	test: 0.2869446	best: 0.2868817 (697)	total: 2.77s	remaining: 36.7s
800:	learn: 0.0159225	test: 0.2868477	best: 0.2868173 (733)	total: 3.11s	remaining: 35.7s
900:	learn: 0.0116849	test: 0.2865292	best: 0.2865167 (899)	total: 3.45s	remaining: 34.8s
1000:	learn: 0.0088985	test: 0.2863613	best: 0.2863464 (980)	total: 3.79s	remaining: 34.1s
1100:	learn: 0